In [1]:
import praw
from textblob import TextBlob
# VADER can be accessed by the NLTK library.
import nltk
# Download the VADAR tool and access it through the NLTK library.
nltk.download("vader_lexicon")
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/liuba/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
reddit = praw.Reddit(client_id='5YD5YW7SySBWEQ',
                     client_secret='aFKHffRIj5J42GIv9VP8PC-jrlw',
                     password='P4SBMH5KGYJgDYm',
                     user_agent='reddit_ninja',
                     username='lcuzacov')

In [3]:
top_posts = reddit.subreddit('Showerthoughts').top('day', limit=5)

In [4]:
# Sentiment analysis function for TextBlob tools
def text_blob_sentiment(review, sub_entries_textblob):
    analysis = TextBlob(review)
    if analysis.sentiment.polarity >= 0.0001:
        if analysis.sentiment.polarity > 0:
            sub_entries_textblob['positive'] += 1
            sub_entries_textblob['total'] += 1
            return 'Positive'

    elif analysis.sentiment.polarity <= -0.0001:
        if analysis.sentiment.polarity <= 0:
            sub_entries_textblob['negative'] += 1
            sub_entries_textblob['total'] += 1
            return 'Negative'
    else:
        sub_entries_textblob['neutral'] += 1
        sub_entries_textblob['total'] += 1
        return 'Neutral'

In [5]:
# create object for VADER sentiment function interaction
sia = SentimentIntensityAnalyzer()

# sentiment analysis function for VADER tool
def nltk_sentiment(review, sub_entries_nltk):
    vs = sia.polarity_scores(review)
    if not vs['neg'] > 0.05:
        if vs['pos'] - vs['neg'] > 0:
            sub_entries_nltk['positive'] += 1
            sub_entries_nltk['total'] += 1
            return 'Positive'
        else:
            sub_entries_nltk['neutral'] += 1
            sub_entries_nltk['total'] += 1
            return 'Neutral'

    elif not vs['pos'] > 0.05:
        if vs['pos'] - vs['neg'] <= 0:
            sub_entries_nltk['negative'] += 1
            sub_entries_nltk['total'] += 1
            return 'Negative'
        else:
            sub_entries_nltk['neutral'] += 1
            sub_entries_nltk['total'] += 1
            return 'Neutral'
    else:
        sub_entries_nltk['neutral'] += 1
        sub_entries_nltk['total'] += 1
        return 'Neutral'


In [6]:
# replication of comment section of reddit post
def replies_of(top_level_comment, count_comment, sub_entries_textblob, sub_entries_nltk):
    if len(top_level_comment.replies) == 0:
        count_comment = 0
        return
    else:
        for num, comment in enumerate(top_level_comment.replies):
            try:
                count_comment += 1
                #print('-' * count_comment)
                text_blob_sentiment(comment.body, sub_entries_textblob)
                nltk_sentiment(comment.body, sub_entries_nltk)
            except:
                continue
            replies_of(comment, count_comment, sub_entries_textblob,sub_entries_nltk)

In [7]:
for submission in top_posts:
        sub_entries_textblob = {'total': 0, 'negative': 0, 'positive' : 0, 'neutral' : 0}
        sub_entries_nltk = {'total': 0, 'negative': 0, 'positive' : 0, 'neutral' : 0}
        print('Title of the post :', submission.title)
        
        text_blob_sentiment(submission.title, sub_entries_textblob)
        nltk_sentiment(submission.title, sub_entries_nltk)
        
        submission_comm = reddit.submission(id=submission.id)
            
        com_count = 0
        for count, top_level_comment in enumerate(submission_comm.comments):
            count_comm = 0
            try :
                text_blob_sentiment(top_level_comment.body, sub_entries_textblob)
                nltk_sentiment(top_level_comment.body, sub_entries_nltk)
                replies_of(top_level_comment, count_comm, sub_entries_textblob, sub_entries_nltk)
            except:
                continue
        print('Over all Sentiment of Topic by TextBlob :', sub_entries_textblob)
        print('Over all Sentiment of Topic by VADER :', sub_entries_nltk)
        print("\n\n\n")

Title of the post : there's a 1 in 7 chance that the meteor that killed the dinosaurs hit on a monday
Over all Sentiment of Topic by TextBlob : {'total': 290, 'negative': 64, 'positive': 87, 'neutral': 139}
Over all Sentiment of Topic by VADER : {'total': 290, 'negative': 55, 'positive': 90, 'neutral': 145}




Title of the post : There’s nothing more irritating than listening to a group of people talk, so inaccurately, about a subject in which you are a profesional and well versed.
Over all Sentiment of Topic by TextBlob : {'total': 402, 'negative': 132, 'positive': 171, 'neutral': 99}
Over all Sentiment of Topic by VADER : {'total': 402, 'negative': 82, 'positive': 110, 'neutral': 210}




Title of the post : Internet Explorer was so slow that it was able to delay its own death for like 10 years or so
Over all Sentiment of Topic by TextBlob : {'total': 86, 'negative': 19, 'positive': 34, 'neutral': 33}
Over all Sentiment of Topic by VADER : {'total': 86, 'negative': 11, 'positive': 3